In [ ]:
import geopandas as gpd
import numpy as np
import plotly.express as px
import plotly.offline as po
from keplergl import KeplerGl
from shapely.geometry import MultiPolygon, Point, Polygon, box

from srai.regionizers import AdministrativeBoundaryRegionizer

## Tesellate Europe

In [ ]:
eu_bbox = box(minx=-10.478556, miny=34.633284672291, maxx=34.597916, maxy=70.096054)
eu_bbox_gdf = gpd.GeoDataFrame({"geometry": [eu_bbox]}, crs="EPSG:4326")

In [ ]:
abr = AdministrativeBoundaryRegionizer(admin_level=2)

In [ ]:
result_gdf = abr.transform(gdf=eu_bbox_gdf)

In [ ]:
result_gdf

In [ ]:
result_gdf.memory_usage(deep=True)

In [ ]:
import topojson as tp

In [ ]:
topo = tp.Topology(result_gdf, 
    prevent_oversimplify=True,
    toposimplify=0.1,  
    # topoquantize=1
    )
topo.to_gdf(winding_order='CCW_CW').plot()

In [ ]:
result_gdf.plot()

In [ ]:
topo.to_gdf().plot()

In [ ]:
topo2 = topo.toposimplify(
    epsilon=0.1,
    simplify_algorithm='vw', 
    simplify_with='simplification', 
    prevent_oversimplify=False
)

In [ ]:
result_gdf_2 = topo2.to_gdf(winding_order='CCW_CW')
result_gdf_2

In [ ]:
topo.to_gdf().plot()

In [ ]:
result_gdf_2.plot()

In [ ]:
k = KeplerGl()
k.add_data(data=result_gdf_2.copy(), name="eu")
k

In [ ]:
result_gdf.head()

In [ ]:
# rgdf = topo.to_gdf()

# def orient_geom(geom):
#     if type(geom) == Polygon:
#         return orient(geom, sign=-1)
#     elif type(geom) == MultiPolygon:
#         return MultiPolygon([orient(g, sign=-1) for g in geom.geoms])


# pl_regions_2d_gdf = gpd.GeoDataFrame(
#     {"geometry": [orient_geom(geom) for geom in region_polys.values()]},
#     index=list(range(len(region_polys))),
#     crs="EPSG:4326",
# )
# rgdf = result_gdf.head(n=20)
# rgdf = topo.to_gdf(winding_order='CCW_CW')
rgdf = topo.to_gdf(winding_order='CW_CCW')
fig = px.choropleth(
    rgdf,
    geojson=rgdf.geometry,
    locations=rgdf.index,
    color=rgdf.index,
    color_continuous_scale=px.colors.sequential.Viridis,
)
# fig.show(renderer="png")

# fig2 = px.scatter_geo(seeds_gdf, lat=seeds_gdf.geometry.y, lon=seeds_gdf.geometry.x)
fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choropleth"))
# fig.add_trace(fig2.data[0])
# fig.update_traces(marker_color="white", marker_size=10, selector=dict(type="scattergeo"))
fig.update_layout(coloraxis_showscale=False)
# fig.update_geos(
#     projection_type="orthographic", projection_rotation_lon=20, projection_rotation_lat=30
# )
minx, miny, maxx, maxy = eu_bbox.bounds
fig.update_geos(
    projection_type="natural earth",
    lataxis_range=[miny - 1, maxy + 1],
    lonaxis_range=[minx - 1, maxx + 1],
)
fig.update_layout(height=500, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()
# fig.show(renderer="png")  # replace with fig.show() to allow interactivity

In [ ]:
from OSMPythonTools.overpass import overpassQueryBuilder

In [ ]:
# "boundary": "administrative", "admin_level": str(admin_level)
query = overpassQueryBuilder(bbox=eu_bbox.bounds, elementType='relation', selector=['"boundary"="administrative"', '"admin_level"="2"'], out='body', includeGeometry=True)

In [ ]:
query

In [ ]:
from OSMPythonTools.overpass import Overpass

In [ ]:
overpass = Overpass()
boundaries = overpass.query(query)

In [ ]:
boundaries.countRelations()

In [ ]:
boundaries.isValid()

In [ ]:
for el in boundaries.elements():
    print(el.id(), el.tags())